# Activity #1: Heat maps
* we'll start with building up a heat map based on some small, randomly generate data
* we'll use this methodology to make our plot interactive & then move on to using "real" data

In [1]:
# lets import our usual stuff
import pandas as pd
import bqplot
import numpy as np
import traitlets
import ipywidgets
%matplotlib inline

In [2]:
# lets start thinking about heatmaps with some random data
data = np.random.random((10, 10))
data
# so we just have a 10 x 10 array here

array([[0.13433743, 0.15167281, 0.42557898, 0.08087435, 0.29428468,
        0.66099415, 0.62826217, 0.15598958, 0.93838148, 0.55438848],
       [0.55304793, 0.38206051, 0.29253455, 0.06593939, 0.20751413,
        0.07709944, 0.33133495, 0.40178809, 0.04262426, 0.66305804],
       [0.99527616, 0.77119589, 0.50395617, 0.61865947, 0.33311249,
        0.57289964, 0.50738764, 0.74388059, 0.49806971, 0.40368   ],
       [0.34687719, 0.53540335, 0.74468011, 0.92367257, 0.04037236,
        0.28846647, 0.3378396 , 0.77048814, 0.94455511, 0.9186086 ],
       [0.15893055, 0.69907509, 0.20789944, 0.46093314, 0.24688182,
        0.75319083, 0.813395  , 0.16504105, 0.6627295 , 0.88129217],
       [0.63288486, 0.6596936 , 0.07905643, 0.86938538, 0.60246052,
        0.37223195, 0.95980439, 0.45499132, 0.28910971, 0.2717295 ],
       [0.80851574, 0.62451461, 0.3239091 , 0.99245018, 0.81384133,
        0.6128713 , 0.00784589, 0.03903341, 0.2618447 , 0.88216266],
       [0.58333663, 0.5710502 , 0.6880431

In [3]:
# lets start by generating a quick heat map

# (1)
# create our first scale of our plot: just a color scale
col_sc = bqplot.ColorScale() 
# now we'll use bqplot's gridheatmap function
#  with our randomly generated data & our scales to 
#  make a heatmap like so:
heat_map = bqplot.GridHeatMap(color = data, 
                              scales = {'color': col_sc})
# put our marks into our figure and lets go!
fig = bqplot.Figure(marks = [heat_map])

# (2) ok, this is fine and all, but lets add some reference for our 
#  color scheme with a colorbar & also lets choose a different 
#  color scheme
col_sc = bqplot.ColorScale(scheme = "Reds")
# lets plot some axes on our plot as well, in this case
#  our axis will be a color bar, vertically on the right
#  of our heatmap
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')
# put it all together and lets take a look!
heat_map = bqplot.GridHeatMap(color = data, 
                              scales = {'color': col_sc})
# generate fig!
fig = bqplot.Figure(marks = [heat_map], axes = [c_ax])

# (3) finally, lets add some axes labels on the x & y axis,
#  we need to add their scales first
# this scale will just count up the boxes in the vertical 
#   & horizontal direction
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()
# add our axes objects
x_ax = bqplot.Axis(scale = x_sc)
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical')
heat_map = bqplot.GridHeatMap(color = data, 
                              scales = {'color': col_sc, 
                                        'row': y_sc,
                                        'column':x_sc})
fig = bqplot.Figure(marks = [heat_map], 
                    axes = [c_ax, y_ax, x_ax])


fig

Figure(axes=[ColorAxis(orientation='vertical', scale=ColorScale(scheme='Reds'), side='right'), Axis(orientatio…

In [4]:
# so, while this indeed a lovely heatmap, it isn't interactive in any way!
#  boo to that!
# Lets start adding in some interactivity

# keep data from last time

# now add scales - colors, x & y
col_sc = bqplot.ColorScale(scheme = "Reds")
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# create axis - for colors, x & y
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc)
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical')

# lets now re-do our heat map & add in some interactivity:
heat_map = bqplot.GridHeatMap(color = data,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, # to make our selection blue
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 0.8})

# stir and combine into 1 figure
fig = bqplot.Figure(marks = [heat_map], 
                    axes = [c_ax, y_ax, x_ax])

fig


Figure(axes=[ColorAxis(orientation='vertical', scale=ColorScale(scheme='Reds'), side='right'), Axis(orientatio…

In [5]:
#  Ok fine, but our selection isn't linked to anything!
#  lets check out what heat_map selected is
heat_map.selected
#  note if I select a different box & re-run this cell,
#  I get out different values


In [6]:
# so now, lets write a little function that links the data value
#  to the selected & lets print this in a little ipywidgets label
mySelectedLabel = ipywidgets.Label()

# (1) 
# lets write our linking function
# there are a few ways to link this,
#  here is a simple way first
def get_data_value(change):
    i,j = heat_map.selected[0]
    v = data[i,j] # grab data value
    mySelectedLabel.value = str(v) # set our label

# (2) this is maybe in-elegant as we are 
#  explicitly calling our origininal heat map!
#  so, lets instead remind ourselves what "change" is here
def get_data_value(change):
    print(change)
    i,j = heat_map.selected[0]
    v = data[i,j] # grab data value
    mySelectedLabel.value = str(v) # set our label
# now we see when we click we get back a whole
#  dictionary of information - if we recall, 
#  "owner" here is our heat_map which "owns" 
# this change.
#  If we want to be able to apply our function to 
#  this or any other heatmap figure we generate,
#  we can re-write the above function as follows:

# (3)
#def get_data_value(change,mylab):
def get_data_value(change):
    #print(change['owner'].selected)
    i,j = change['owner'].selected[0]
    v = data[i,j] # grab data value
    mySelectedLabel.value = str(v) # set our label
    #mylab.value = str(v) # set our label
# so, this now is applied to any map that we choose to input
    
# regenerate our heatmap to use in our fig canvas
heat_map = bqplot.GridHeatMap(color = data,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 0.8})
    
# make sure we check out     
heat_map.observe(get_data_value, 'selected')
#heat_map.observe(self, mySelectedLabel)
fig = bqplot.Figure(marks = [heat_map], 
                    axes = [c_ax, y_ax, x_ax])

ipywidgets.VBox([mySelectedLabel, fig])
#fig

# Activity #2: Preliminary dashboarding
* we'll use a random dataset to explore how to make dashboard-like plots that change when things are updated

In [7]:
# now lets move on to making a preliminary 
#dashboard for multi-dimensional datasets
#  lets first start with some randomly generated data again

In [8]:
data = np.random.random((10, 10,20))
data

data.shape

data[0,0,:]
# we can see that no instead of 1 value, each "i,j" component
#  has an array of values

# lets start building up linked plots
#  first, lets re-do our plot above with our label printing
#  out the sum along this 5-d array

# now add scales - colors, x & y
col_sc = bqplot.ColorScale(scheme = "Reds")
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# create axis - for colors, x & y
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc)
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical')

# create label again
mySelectedLabel = ipywidgets.Label()

def get_data_value(change):
    i,j = change['owner'].selected[0]
    # if we run with this, our label is the 20 elements
    #v = data[i,j] # grab data value
    # but,lets sum instead
    v = data[i,j].sum() # grab data value
    mySelectedLabel.value = str(v) # set our label
# so, this now is applied to any map that we choose to input
    
    
    
# regenerate our heatmap to use in our fig canvas
# now, we want to plot the sum along our 3rd axis as well, 
#  so, lets do this with "np.sum" along our 3rd axis
heat_map = bqplot.GridHeatMap(color = np.sum(data,axis=2),
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 0.8})
    
# make sure we check out     
heat_map.observe(get_data_value, 'selected')
#heat_map.observe(self, mySelectedLabel)
fig = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax])

#(1)
#ipywidgets.VBox([mySelectedLabel, fig])

# (2)
# now, lets generate another figure that just plots the histogram of values in our 3rd axis
x_sch = bqplot.LinearScale()
y_sch = bqplot.LinearScale()
x_axh = bqplot.Axis(scale = x_sch, label = 'Sum of 3rd axis')
y_axh = bqplot.Axis(scale = y_sch, 
                    orientation = 'vertical', 
                    label='Frequency')

hist = bqplot.Hist(sample = data[0,0,:],
                    opacity = 0.1, 
                   normalized = False, # normalized=False means we get counts in each bin
                    scales = {'sample': x_sch, 'count': y_sch},
                  bins = 5)
figh = bqplot.Figure(marks = [hist], axes = [x_axh, y_axh])
# ok, so side by side plots, but nothing updates!

#(3) so, we have to update what our heatmap has access to as 
#  far as being able to update both the label *AND* the 
# histogram's data
def get_data_value2(change):
    i,j = change['owner'].selected[0]
    # if we run with this, our label is the 20 elements
    #v = data[i,j] # grab data value
    # but,lets sum instead
    v = data[i,j].sum() # grab data value
    mySelectedLabel.value = str(v) # set our label
    hist.sample = data[i,j]
    #print(data[i,j])
heat_map.observe(get_data_value2, 'selected')
# note here now the heat_map is in a sense "driving" our
# changes.
# *** DO EXAMPLE OF BACK AND FORTH ***


ipywidgets.VBox([mySelectedLabel, ipywidgets.HBox([fig,figh])] )


# Activity #3: Dashboarding with "real" data
* now we'll move onto the UFO dataset and start messing around with creating a dashboard for this dataset

In [9]:
# lets start by loading the UFO dataset
ufos = pd.read_csv("/Users/jillnaiman/Downloads/ufo-scrubbed-geocoded-time-standardized-00.csv",
                  names = ["date", "city", "state", "country",
                          "shape", "duration_seconds", "duration",
                          "comment", "report_date", 
                           "latitude", "longitude"],
                  parse_dates = ["date", "report_date"])

## Aside: downsampling
* some folks reported having a tough time with interactivity of scatter plots with the UFO dataset
* here we'll quickly go over some methods of downsampling that can be applied to decrease the size of our dataset

In [10]:
# you'll see the above takes a good long time to load on my computer
# the length of the dataset is quite large:
len(ufos)

80332

In [11]:
# 80,000!  So, to speed up our interactivity, we can 
#  randomly sample this dataset for plotting purposes
# lets down sample to 1000 samples:
nsamples = 1000
#nsamples = 5000
downSampleMask = np.random.randint(0,len(ufos)-1,nsamples)
downSampleMask
# so, downsample mask is now a list of random indicies for 
#  the UFO dataset

array([79364, 42257,  6717, 78121, 59184,  6677, 24651, 53255, 10262,
       41983,  8675, 49297,  6365, 11800,  8974, 44620, 39846, 23727,
         435, 54986,  2209, 73634, 67466, 29746, 12648, 76062, 63150,
       58364, 36472, 12272, 29462, 34928, 13035, 42961, 37580, 23010,
       51422, 47429, 42671, 24371, 35638, 72280, 16865, 77994, 61954,
       16203, 39330, 53216, 42201, 14688, 21248, 51701, 15344,   275,
       34914, 57051, 12899,    53,  2708, 34907, 68727, 19621, 43627,
       40370, 30956, 55354, 48943, 33512, 70108,  3206, 69431,  7104,
       13887, 15763, 17085, 32491, 40109, 79182, 44875, 49120, 29250,
       14712, 32488, 16756, 23992, 68255, 66795,  8139, 28793, 75991,
       25686, 47964, 46005, 36963, 33603, 12966, 31474, 44544, 34334,
       19713, 71198, 63525, 36845, 18205, 43285, 25725, 44119,  5593,
       15917, 42619, 41497, 57169, 17076,  8382,  8028, 67528, 32208,
       45290, 17080, 67871, 61678, 15207,  4497, 12192, 10419,  4865,
       12475, 43001,

In [12]:
# the above doesn't disclude repeats, but we can take 
#  care of this with a different call:
downSampleMask = np.random.choice(range(len(ufos)-1), 
                                  nsamples, replace=False)

In [13]:
# lets update:
ufosDS = ufos.loc[downSampleMask]
len(ufosDS)
# so much shorter

1000

In [14]:
# we can also see that this is saved as a dataframe:
ufosDS

,date,city,state,country,shape,duration_seconds,duration,comment,report_date,latitude,longitude
18135,2008-12-14 02:15:00,grants,nm,us,unknown,1800.0,30min still going,2 orange flashing lights with 2 different colo...,2009-01-10,35.147222,-107.850833
23212,2005-12-07 00:50:00,marysville,wa,us,NaN,1.0,1 second,Blue light impact - Marysville,2005-12-16,48.051944,-122.175833
63868,2012-07-08 22:50:00,auckland (new zealand),NaN,NaN,fireball,300.0,1-5 minutes,x2 yellowish/white spheres seen in clear night...,2012-12-20,-36.848460,174.763332
8259,2001-01-11 21:40:00,rockford,il,us,light,900.0,15 mins,same as above,2001-02-18,42.271111,-89.093889
60182,2009-07-27 03:30:00,flower mound,tx,us,light,35.0,35 seconds,Unknown lights hovering above store.,2011-01-31,33.014444,-97.096667
...,...,...,...,...,...,...,...,...,...,...,...
48331,2011-06-13 22:21:00,leavenworth,wa,us,triangle,360.0,6 minutes,Bright triangular shape changing direction and...,2011-06-20,47.596389,-120.660278
49526,2004-06-16 22:27:00,bristol,tn,us,other,240.0,4 minutes,MUFON/TN REPORT: Two joined objects spotted d...,2004-06-18,36.595000,-82.188889
37004,2006-04-15 17:15:00,wilmington (suburbs),de,us,sphere,60.0,1 minute,Brilliant sphere floats above the Wilmington S...,2006-07-16,39.745833,-75.546944
55658,1976-07-12 18:01:00,warwick,ri,us,cigar,1800.0,30 min,I saw a fast moving&#44 brightly lit&#44 huge ...,2009-03-19,41.700000,-71.416667


In [15]:
# lets make a super quick scatter plot to remind ourselves what this looks like:
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

x_ax = bqplot.Axis(scale = x_sc, label='Longitude')
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', 
                   label='Latitude')

#(1)
#scatters = bqplot.Scatter(x = ufosDS['longitude'],
#                          y = ufosDS['latitude'],
#                          scales = {'x': x_sc, 'y': y_sc})

# (2) recall we can also color by things like duration
c_sc = bqplot.ColorScale()
#c_ax = bqplot.ColorAxis(scale = c_sc, label='Duration in sec', orientation = 'vertical', side = 'right')
#scatters = bqplot.Scatter(x = ufosDS['longitude'],
#                          y = ufosDS['latitude'],
#                          color=ufosDS['duration_seconds'],
#                          scales = {'x': x_sc, 'y': y_sc, 'color':c_sc})

# (3) again, we recall that there is a large range in durations, so 
#  it makes sense that we have a muted color pattern - we want 
# to use a log colorscale
# with bqplot we can do this with:
c_ax = bqplot.ColorAxis(scale = c_sc, label='log(sec)', 
                        orientation = 'vertical', side = 'right')
scatters = bqplot.Scatter(x = ufosDS['longitude'],
                          y = ufosDS['latitude'],
                          color=np.log10(ufosDS['duration_seconds']),
                          scales = {'x': x_sc, 'y': y_sc, 'color':c_sc})

fig = bqplot.Figure(marks = [scatters], axes = [x_ax, y_ax, c_ax]) 
fig

Figure(axes=[Axis(label='Longitude', scale=LinearScale()), Axis(label='Latitude', orientation='vertical', scal…

In [16]:
# now we are going to use our heatmap idea to plot this data again
#  note this will shmear out a lot of the nice map stuff we see above
#  don't worry!  We'll talk about making maps in the next class or so

# what should we color by?  lets do by duration

# to get this to work with our heatmap, we're going 
#  to have to do some rebinning
#  right now, our data is all in 1 long list
# we need to rebin things in a 2d histogram where 
# the x axis is long & y is lat

# ***START WITH 10 EACH**
nlong = 20 
nlat = 20

#(1)
hist2d, long_edges, lat_edges = np.histogram2d(ufos['longitude'], 
                                               ufos['latitude'], 
                                               weights=ufos['duration_seconds'], 
                                              bins=[nlong,nlat])
# this returns the TOTAL duration of ufo events in each bin
hist2d

# (2)
# to do the average duration in each bin we can do:
hist2d, long_edges, lat_edges = np.histogram2d(ufos['longitude'], 
                                               ufos['latitude'], 
                                               weights=ufos['duration_seconds'],
                                              normed=True, 
                                              bins = [nlong,nlat])
hist2d

# (3) ok, lets go back to total duration
hist2d, long_edges, lat_edges = np.histogram2d(ufos['longitude'], 
                                               ufos['latitude'], 
                                               weights=np.log10(ufos['duration_seconds']),
                                              bins = [nlong,nlat])

# note that the sizes of the edges & the hist are different:
hist2d.shape, long_edges.shape, lat_edges.shape

# this is becuase the edges are bin edges, not centers
# to get bin centers we can do:
# lets do some fancy in-line forloops
long_centers = [(long_edges[i]+long_edges[i+1])*0.5 for i in range(len(long_edges)-1)]
lat_centers = [(lat_edges[i]+lat_edges[i+1])*0.5 for i in range(len(lat_edges)-1)]
long_centers, lat_centers

# (4) note: we might want to control where our bins are, we can do this by 
#  specifying bin edges ourselves
long_bins = np.linspace(-150, 150, nlong+1)
lat_bins = np.linspace(-40, 70, nlat+1)
long_bins, long_bins.shape
lat_bins, lat_bins.shape


hist2d, long_edges, lat_edges = np.histogram2d(ufos['longitude'], 
                                               ufos['latitude'], 
                                               weights=ufos['duration_seconds'],
                                              bins = [long_bins,lat_bins])

# this is becuase the edges are bin edges, not centers
long_centers = [(long_edges[i]+long_edges[i+1])*0.5 for i in range(len(long_edges)-1)]
lat_centers = [(lat_edges[i]+lat_edges[i+1])*0.5 for i in range(len(lat_edges)-1)]

# (5)
# again, we want to take the log scale of things
#  we're going to do this by taking the log of hist2d
#  but there are some zero values in this hsitogram
# if we just take the log we get -inf
np.log10(hist2d)
# this can mess up our color scheme mapping

# (6) so we are going to "trick" our color scheme like so
hist2d[hist2d <= 0] = np.nan # set zeros to NaNs
# then take log
hist2d = np.log10(hist2d)
hist2d

# (7) finally, our histogram is actually
#  transposed - this is just how numpy outputs it,
# lets put the world right side up with:
hist2d = hist2d.T

/Users/jillnaiman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log10


In [17]:
# now that we have all that fancy binning out of the way, 
#  lets proceed as normal:

# add scales - colors, x & y
col_sc = bqplot.ColorScale(scheme="RdPu", 
                           min=np.nanmin(hist2d), 
                           max=np.nanmax(hist2d))
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

# create axis - for colors, x & y
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')#,
                       #label='log(sec)')
x_ax = bqplot.Axis(scale = x_sc, label='Longitude')
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', 
                   label = 'Latitude')

heat_map = bqplot.GridHeatMap(color = hist2d,
                              row = lat_centers, 
                              column = long_centers,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 1.0})

#***GO BACK AND PLAY WITH BIN SIZES***

# (2) lets add a label again to pritn duration
# create label again
mySelectedLabel = ipywidgets.Label()
def get_data_value(change):
    i,j = change['owner'].selected[0]
    v = hist2d[i,j] # grab data value
    mySelectedLabel.value = 'Total duration in log(sec) = ' + str(v) # set our label
        
        
        
        
# make sure we check out     
heat_map.observe(get_data_value, 'selected')


fig = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax])

#(1)
#fig

#(2)
ipywidgets.VBox([mySelectedLabel,fig])

In [18]:
# ok, now lets build up our dashboard 
# again to also show how the duration of UFO sitings in each 
# selected region changes with year

# we'll do this with the same methodology we applied before
#  **copy paste above***

# (1)

# (I) For the heatmap
# add scales - colors, x & y
col_sc = bqplot.ColorScale(scheme="RdPu", 
                           min=np.nanmin(hist2d), 
                           max=np.nanmax(hist2d))
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# create axis - for colors, x & y
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc, label='Longitude')
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', 
                   label = 'Latitude')

heat_map = bqplot.GridHeatMap(color = hist2d,
                              row = lat_centers, 
                              column = long_centers,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 1.0})

fig = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax])

# (II) Scatter plot
# scales & ax in usual way
import datetime as dt
x_scl = bqplot.DateScale(min=dt.datetime(1950,1,1),max=dt.datetime(2020,1,1)) # note: for dates on x-axis
y_scl = bqplot.LogScale()
ax_xcl = bqplot.Axis(label='Date', scale=x_scl)
ax_ycl = bqplot.Axis(label='Duration in Sec', scale=y_scl, 
                    orientation='vertical', side='left')
# for the lineplot of duration in a region as a function of year
# lets start with a default region & year
i,j = 0,0
longs = [long_edges[i], long_edges[i+1]]
lats = [lat_edges[j],lat_edges[j+1]]
region_mask = ( (ufos['latitude'] >= lats[0]) & (ufos['latitude']<=lats[1]) &\
                (ufos['longitude'] >= longs[0]) & (ufos['longitude']<=longs[1]) )

# we can see this selects for the upper right point of our heatmap
lats, longs, ufos['latitude'][region_mask]

# lets plot the durations as a function of year there
duration_scatt = bqplot.Scatter(x = ufos['date'][region_mask],
                               y = ufos['duration_seconds'][region_mask], 
                              scales={'x':x_scl, 'y':y_scl})

fig_dur = bqplot.Figure(marks = [duration_scatt], axes = [ax_xcl, ax_ycl])

# create label again
mySelectedLabel = ipywidgets.Label()
def get_data_value(change):
    i,j = change['owner'].selected[0]
    v = hist2d[i,j] # grab data value
    mySelectedLabel.value = 'Total duration in log(sec) = ' + str(v) # set our label
        
# make sure we connect to heatmap     
#heat_map.observe(get_data_value, 'selected')

# (2) now again, we want our scatter plot to react to changes 
#  to what we've selected so:
def get_data_value2(change):
    i,j = change['owner'].selected[0]
    v = hist2d[i,j] # grab data value
    mySelectedLabel.value = 'Total duration in log(sec) = ' + str(v) # set our label
    # note!! i & j are swapped here to machup with hist & selection 
    longs = [long_edges[j], long_edges[j+1]]
    lats = [lat_edges[i],lat_edges[i+1]]
    region_mask = ( (ufos['latitude'] >= lats[0]) & (ufos['latitude']<=lats[1]) &\
                (ufos['longitude'] >= longs[0]) & (ufos['longitude']<=longs[1]) )
    duration_scatt.x = ufos['date'][region_mask]
    duration_scatt.y = ufos['duration_seconds'][region_mask]
    #print(i,j)
    #print(longs,lats)
    #print(ufos['date'][region_mask])
# make sure we connect to heatmap     
heat_map.observe(get_data_value2, 'selected')


ipywidgets.VBox([mySelectedLabel, ipywidgets.HBox([fig,fig_dur])])
# note that when I select a deep purple place, my scatter plot is 
#  very laggy, this makes me think we should do this with a 
#  histogram/bar type plot


In [21]:
# (I) For the heatmap
# add scales - colors, x & y
col_sc = bqplot.ColorScale(scheme="RdPu", 
                           min=np.nanmin(hist2d), 
                           max=np.nanmax(hist2d))
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# create axis - for colors, x & y
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc, label='Longitude')
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', 
                   label = 'Latitude')

heat_map = bqplot.GridHeatMap(color = hist2d,
                              row = lat_centers, 
                              column = long_centers,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 1.0})

fig = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax])

# (II) Bar plot
# scales & ax in usual way
x_scl = bqplot.LinearScale() # note we are back to linears
y_scl = bqplot.LinearScale()
ax_xcl = bqplot.Axis(label='Date', scale=x_scl)
ax_ycl = bqplot.Axis(label='Total duration in Sec', scale=y_scl, 
                    orientation='vertical', side='left')
# for the lineplot of duration in a region as a function of year
# lets start with a default region & year
i,j = 0,0
longs = [long_edges[i], long_edges[i+1]]
lats = [lat_edges[j],lat_edges[j+1]]
region_mask = ( (ufos['latitude'] >= lats[0]) & (ufos['latitude']<=lats[1]) &\
                (ufos['longitude'] >= longs[0]) & (ufos['longitude']<=longs[1]) )

# we can see this selects for the upper right point of our heatmap
lats, longs, ufos['latitude'][region_mask]

# lets plot the durations as a function of year there
ufos['year'] = ufos['date'].dt.year
dur, dur_edges = np.histogram(ufos['year'][region_mask],
                              weights=ufos['duration_seconds'][region_mask],
                              bins=10)
# like before with our histograms
dur_centers = [(dur_edges[i]+dur_edges[i+1])*0.5 for i in range(len(dur_edges)-1)]
# make histogram by hand, weighting by duration
duration_hist = bqplot.Bars(x=dur_centers, y=dur, 
                          scales={'x':x_scl, 'y':y_scl})
fig_dur = bqplot.Figure(marks = [duration_hist], axes = [ax_xcl, ax_ycl])


#  to what we've selected so:
def get_data_value(change):
    i,j = change['owner'].selected[0]
    v = hist2d[i,j] # grab data value
    mySelectedLabel.value = 'Total duration in log(sec) = ' + str(v) # set our label
    # note!! i & j are swapped here to machup with hist & selection 
    longs = [long_edges[j], long_edges[j+1]]
    lats = [lat_edges[i],lat_edges[i+1]]
    region_mask = ( (ufos['latitude'] >= lats[0]) & (ufos['latitude']<=lats[1]) &\
                (ufos['longitude'] >= longs[0]) & (ufos['longitude']<=longs[1]) )
    if len(ufos['year'][region_mask]) > 0:
        dur, dur_edges = np.histogram(ufos['year'][region_mask],
                                      weights=ufos['duration_seconds'][region_mask],
                                      bins=10)
        dur_centers = [(dur_edges[i]+dur_edges[i+1])*0.5 for i in range(len(dur_edges)-1)]
        duration_hist.x = dur_centers
        duration_hist.y = dur
    #else:
    #    duration_hist.x = np.arange(10); duration_hist.y = np.zeros(10)
# make sure we connect to heatmap     
heat_map.observe(get_data_value, 'selected')

fig.layout.min_width = '500px'
fig_dur.layout.min_width = '700px'
plots = ipywidgets.HBox([fig,fig_dur])
myout = ipywidgets.VBox([mySelectedLabel, plots])
myout

## Might not get to this...

In [20]:
col_sc = bqplot.ColorScale(scheme="RdPu", 
                           min=np.nanmin(hist2d), 
                           max=np.nanmax(hist2d))
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# create axis - for colors, x & y
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc, label='Longitude')
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', 
                   label = 'Latitude')

heat_map = bqplot.GridHeatMap(color = hist2d,
                              row = lat_centers, 
                              column = long_centers,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 1.0})

fig = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax])

# (II) Bar plot for durations thorugh the years
# scales & ax in usual way
x_scl = bqplot.LinearScale() # note we are back to linears
y_scl = bqplot.LinearScale()
ax_xcl = bqplot.Axis(label='Date', scale=x_scl)
ax_ycl = bqplot.Axis(label='Total duration in Sec', scale=y_scl, 
                    orientation='vertical', side='left')
# for the lineplot of duration in a region as a function of year
# lets start with a default region & year
i,j = 0,0
longs = [long_edges[i], long_edges[i+1]]
lats = [lat_edges[j],lat_edges[j+1]]
region_mask = ( (ufos['latitude'] >= lats[0]) & (ufos['latitude']<=lats[1]) &\
                (ufos['longitude'] >= longs[0]) & (ufos['longitude']<=longs[1]) )

# we can see this selects for the upper right point of our heatmap
lats, longs, ufos['latitude'][region_mask]

# lets plot the durations as a function of year there
ufos['year'] = ufos['date'].dt.year
dur, dur_edges = np.histogram(ufos['year'][region_mask],
                              weights=ufos['duration_seconds'][region_mask],
                              bins=10)
# like before with our histograms
dur_centers = [(dur_edges[i]+dur_edges[i+1])*0.5 for i in range(len(dur_edges)-1)]
# make histogram by hand, weighting by duration
duration_hist = bqplot.Bars(x=dur_centers, y=dur, 
                          scales={'x':x_scl, 'y':y_scl})
fig_dur = bqplot.Figure(marks = [duration_hist], axes = [ax_xcl, ax_ycl])

# (III) histogram for shape
x_ord = bqplot.OrdinalScale()
y_ord = bqplot.LinearScale()
ax_xord = bqplot.Axis(label='Shape', scale=x_ord)
ax_yord = bqplot.Axis(label='Freq', scale=y_ord,
                     orientation='vertical',
                     side='left')

# histogram using pandas
hist_ord = bqplot.Bars(x=ufos['shape'][region_mask].unique(),
                       y=ufos['shape'][region_mask].value_counts(),
                     scales={'x':x_ord, 'y':y_ord})
fig_shape = bqplot.Figure(marks=[hist_ord], axes=[ax_xord,ax_yord])

#  to what we've selected so:
def get_data_value(change):
    i,j = change['owner'].selected[0]
    v = hist2d[i,j] # grab data value
    mySelectedLabel.value = 'Total duration in log(sec) = ' + str(v) # set our label
    # note!! i & j are swapped here to machup with hist & selection 
    longs = [long_edges[j], long_edges[j+1]]
    lats = [lat_edges[i],lat_edges[i+1]]
    region_mask = ( (ufos['latitude'] >= lats[0]) & (ufos['latitude']<=lats[1]) &\
                (ufos['longitude'] >= longs[0]) & (ufos['longitude']<=longs[1]) )
    dur, dur_edges = np.histogram(ufos['year'][region_mask],
                                  weights=ufos['duration_seconds'][region_mask],
                                  bins=10)
    dur_centers = [(dur_edges[i]+dur_edges[i+1])*0.5 for i in range(len(dur_edges)-1)]
    duration_hist.x = dur_centers
    duration_hist.y = dur
    # also update shapes
    #print(ufos['shape'][region_mask])
    hist_ord.x = ufos['shape'][region_mask].unique()
    hist_ord.y = ufos['shape'][region_mask].value_counts()
# make sure we connect to heatmap     
heat_map.observe(get_data_value, 'selected')

# lets make all the sizes look nice
fig_dur.layout.max_width = '400px'
fig_dur.layout.max_height= '300px'
fig_shape.layout.max_width = '400px'
fig_shape.layout.max_height= '300px'
fig.layout.min_width = '800px' # add to both

# dhange layout
ipywidgets.VBox([mySelectedLabel, ipywidgets.HBox([fig,ipywidgets.VBox([fig_shape,fig_dur])])])
#myout = ipywidgets.VBox([mySelectedLabel,
#                ipywidgets.HBox([fig_shape,fig_dur]),
#                fig])
#myout

In [37]:
data2 = np.random.random((10, 10))

# tester
def on_selected(change):
    print(change['owner'].selected)

# recopy our scales and axis for posterity:
col_sc = bqplot.ColorScale(scheme = "Reds")
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()
# create axis - for colors, x & y
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc)
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical')

# lets now re-do our heat map & add in some interactivity:
heat_map = bqplot.GridHeatMap(color = data2,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'}, # make interactive on click of each box
                              anchor_style = {'fill':'blue'}, # to make our selection blue
                              selected_style = {'opacity': 1.0}, # make 100% opaque if box is selected
                              unselected_style = {'opacity': 0.8}) # make a little see-through if not

## THIS IS ALL WE HAVE ADDED!
heat_map.observe(on_selected, 'selected')

# stir and combine into 1 figure
fig = bqplot.Figure(marks = [heat_map], 
                    axes = [c_ax, y_ax, x_ax])

fig

Figure(axes=[ColorAxis(orientation='vertical', scale=ColorScale(scheme='Reds'), side='right'), Axis(orientatio…